# Quickstart

Import the `kif_lib` namespace:

In [1]:
from kif_lib import *

Import the Wikidata vocabulary module `wd`:

In [2]:
from kif_lib.vocabulary import wd

Create a KIF store pointing to the official Wikidata query service:

In [3]:
kb = Store('wikidata')

Fetch and print three statements about Brazil:

In [4]:
it = kb.filter(subject=wd.Brazil, limit=3)
for stmt in it:
    display(stmt)

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Paraguay](http://www.wikidata.org/entity/Q733))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [wd:Q142](http://www.wikidata.org/entity/Q142))))

## Filters

The `kb.filter(...)` call searches for statements in `kb` whose structure matches the restrictions `...`.

### Basic filters

We can filter statements by any combination of *subject*, *property*, and *value*:

In [5]:
display(next(kb.filter(subject=wd.Brazil, property=wd.official_website)))
display(next(kb.filter(property=wd.official_website, value=IRI('https://www.ibm.com/'))))
display(next(kb.filter(value=Quantity('78.046950192', unit=wd.dalton))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [official website](http://www.wikidata.org/entity/P856)) [https://www.gov.br](https://www.gov.br)))

(**Statement** (**Item** [IBM](http://www.wikidata.org/entity/Q37156)) (**ValueSnak** (**Property** [official website](http://www.wikidata.org/entity/P856)) [https://www.ibm.com/](https://www.ibm.com/)))

(**Statement** (**Item** [wd:Q83057204](http://www.wikidata.org/entity/Q83057204)) (**ValueSnak** (**Property** [mass](http://www.wikidata.org/entity/P2067)) (**Quantity** 78.046950192 (**Item** [dalton](http://www.wikidata.org/entity/Q483261)))))

We can also match statements having an *some* (unknown) value:

In [6]:
next(kb.filter(snak=wd.date_of_birth.some_value()))

(**Statement** (**Item** [wd:Q7262](http://www.wikidata.org/entity/Q7262)) (**SomeValueSnak** (**Property** [date of birth](http://www.wikidata.org/entity/P569))))

Or *no* value:

In [7]:
next(kb.filter(snak=wd.date_of_death.no_value()))

(**Statement** (**Item** [wd:Q124967648](http://www.wikidata.org/entity/Q124967648)) (**NoValueSnak** (**Property** [date of death](http://www.wikidata.org/entity/P570))))

### Indirect ids

We can use properties to identify (indirectly) the subject, property, or value of a statement.

For instance, the following filter matches the statements whose subject is a country and value is a person:

In [8]:
next(kb.filter(subject=wd.instance_of(wd.country_), value=wd.instance_of(wd.continent_)))

(**Statement** (**Item** [wd:Q1049](http://www.wikidata.org/entity/Q1049)) (**ValueSnak** (**Property** [continent](http://www.wikidata.org/entity/P30)) (**Item** [wd:Q15](http://www.wikidata.org/entity/Q15))))

Properties can also be identified indirectly.

For instance, the following filter matches statements whose property is equivalent to Schema.org's "weight":

In [9]:
next(kb.filter(property=wd.equivalent_property('https://schema.org/weight')))

(**Statement** (**Item** [wd:Q38824](http://www.wikidata.org/entity/Q38824)) (**ValueSnak** (**Property** [mass](http://www.wikidata.org/entity/P2067)) (**Quantity** 17 (**Item** [kilogram](http://www.wikidata.org/entity/Q11570)))))

### And-ing and Or-ing

In [10]:
#import logging
#logging.basicConfig(level=logging.DEBUG)
for stmt in kb.filter(subject=(wd.instance_of(wd.country_)&wd.continent(wd.South_America))|wd.Adam, limit=100):
    display(stmt)

(**Statement** (**Item** [Adam](http://www.wikidata.org/entity/Q70899)) (**NoValueSnak** (**Property** [father](http://www.wikidata.org/entity/P22))))

(**Statement** (**Item** [Adam](http://www.wikidata.org/entity/Q70899)) (**NoValueSnak** (**Property** [mother](http://www.wikidata.org/entity/P25))))

(**Statement** (**Item** [Adam](http://www.wikidata.org/entity/Q70899)) (**ValueSnak** (**Property** [child](http://www.wikidata.org/entity/P40)) (**Item** [wd:Q313421](http://www.wikidata.org/entity/Q313421))))

(**Statement** (**Item** [Adam](http://www.wikidata.org/entity/Q70899)) (**ValueSnak** (**Property** [child](http://www.wikidata.org/entity/P40)) (**Item** [wd:Q9639242](http://www.wikidata.org/entity/Q9639242))))

(**Statement** (**Item** [Adam](http://www.wikidata.org/entity/Q70899)) (**ValueSnak** (**Property** [child](http://www.wikidata.org/entity/P40)) (**Item** [wd:Q9641412](http://www.wikidata.org/entity/Q9641412))))

...

(**Statement** (**Item** [wd:Q21203](http://www.wikidata.org/entity/Q21203)) (**ValueSnak** (**Property** [highest point](http://www.wikidata.org/entity/P610)) (**Item** [wd:Q594885](http://www.wikidata.org/entity/Q594885))))

(**Statement** (**Item** [wd:Q21203](http://www.wikidata.org/entity/Q21203)) (**ValueSnak** (**Property** [part of](http://www.wikidata.org/entity/P361)) (**Item** [wd:Q25227](http://www.wikidata.org/entity/Q25227))))

(**Statement** (**Item** [wd:Q21203](http://www.wikidata.org/entity/Q21203)) (**ValueSnak** (**Property** [part of](http://www.wikidata.org/entity/P361)) (**Item** [wd:Q5317255](http://www.wikidata.org/entity/Q5317255))))

(**Statement** (**Item** [wd:Q21203](http://www.wikidata.org/entity/Q21203)) (**ValueSnak** (**Property** [member of](http://www.wikidata.org/entity/P463)) (**Item** [wd:Q8475](http://www.wikidata.org/entity/Q8475))))

(**Statement** (**Item** [wd:Q21203](http://www.wikidata.org/entity/Q21203)) (**ValueSnak** (**Property** [located in time zone](http://www.wikidata.org/entity/P421)) (**Item** [wd:Q5762](http://www.wikidata.org/entity/Q5762))))